### Kaggle ML course team battle 01 for baseline classification

#### 1. load data

In [ ]:
import sklearn
from sklearn.datasets import load_files

# training
datapath = r'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/training_dataset_v1/training_dataset'
mrdataset = load_files(datapath, shuffle=True)

# testing
import pandas as pd
test = pd.read_csv('C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/test_dataset.csv', header=None)

# submission
sub = pd.read_csv('C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission.csv')

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

#### 2. baseline_model function

In [ ]:
from sklearn.model_selection import cross_val_score

def baseline_model(model, vector, mrdataset, test , sub):
    performance = cross_val_score(model, tfidf_vectorizer.fit_transform(mrdataset.data), mrdataset.target, 
                                 cv=10, scoring='accuracy')
    print("avg-accuracy: {0}".format(performance.mean()))

    # test.head()
    x_train = tfidf_vectorizer.fit_transform(mrdataset.data)
    y_train = mrdataset.target
    x_test = tfidf_vectorizer.transform(test.iloc[:,1].values.astype('U'))

    model.fit(x_train,y_train)
    y_test = model.predict(x_test)

    sub['Label'] = y_test
    sub['Label'].replace([0,1],['neg','pos'], inplace=True)
    
    return sub
    

#### 3. Multinomial NB

In [17]:
# NB
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
NB = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_NB_ans.csv'
NB.to_csv(path, index=0)

avg-accuracy: 0.8647600000000001


#### 4. SVM

In [ ]:
# SVM
from sklearn import svm
model = svm.SVC(kernel='linear')
svm = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_svm_ans.csv'
svm.to_csv(path, index=0)

#### 4-1. SVM 調參

In [ ]:
# 參數調起來!!! 
from sklearn.model_selection import GridSearchCV
from sklearn import svm

x_train = tfidf_vectorizer.fit_transform(mrdataset.data)
y_train = mrdataset.target
x_test = tfidf_vectorizer.transform(test.iloc[:,1].values.astype('U'))

parameters = {'kernel':('linear', 'rbf'),'C':[1,2,4],'gamma':[0.125, 0.25, 0.5, 1, 2, 4]}
model = svm.SVC()
gs_clf = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf.fit(tfidf_vectorizer.fit_transform(tfidf_vectorizer.fit_transform(mrdataset.data)[:400]),
           mrdataset.target[:400])
performance = cross_val_score(gs_clf.best_estimator_, tfidf_vectorizer.fit_transform(mrdataset.data), mrdataset.target, 
                             cv=10, scoring='accuracy')
print("avg-accuracy: {0}".format(performance.mean()))
print('The parameters of the best model are :')
print(gs_clf.best_params_)


y_test = gs_clf.predict(x_test)

sub['Label'] = y_test
sub['Label'].replace([0,1],['neg','pos'], inplace=True)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_GS_ans.csv'
sub.to_csv(path, index=0)

#### 5. LR

In [ ]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
LR = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_LR_ans.csv'
LR.to_csv(path, index=0)

#### 6. RF

In [ ]:
# RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
RF = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_RF_ans.csv'
RF.to_csv(path, index=0)

#### 7. kNN

In [ ]:
# kNN
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
kNN = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_kNN_ans.csv'
kNN.to_csv(path, index=0)

#### 8. DT

In [ ]:
# DT
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
DT = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_DT_ans.csv'
DT.to_csv(path, index=0)

#### 9. XGBoost

In [ ]:
# XGboost
from xgboost import XGBClassifier
model = XGBClassifier()
XGB = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_XGB_ans.csv'
XGB.to_csv(path, index=0)

####10. RBF

In [ ]:
#RBF
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
svc = SVC(kernel='rbf', gamma=1.0)
def RBFKernelSVC(gamma=1.0):
    return Pipeline([
        ('std_scaler', StandardScaler()),
        ('svc', SVC(kernel='rbf', gamma=gamma))
    ])
#model = StandardScaler()
#RBF = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

#path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_RBF_ans.csv'
#RBF.to_csv(path, index=0)

In [ ]:
###DNN
import os
import numpy as np
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Activation, Dense
from keras.layers.embeddings import Embedding
top_words = 5000
max_words = 500
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#DNN=baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

#path = 'C:/Users/nicoleshih/Downloads/tmu-ml-course-team-battle-1/submission_DNN_ans.csv'
#DNN.to_csv(path, index=0)